In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt


In [2]:
# sql query
query_str='''
select passenger_count 
      ,pickup_datetime  
      ,pickup_longitude
      ,pickup_latitude
      ,dropoff_longitude
      ,dropoff_latitude
      ,fare_amount 
      from [dbo].[nyctaxi_sample]
'''
servername = 'LNOR010941'
db = 'NYCTaxi_Sample'
conn_str = 'Driver=SQL Server;Server=' + servername + \
    ';Database=' + db + ';Trusted_Connection=True;'
cnxn = pyodbc.connect(conn_str)
#cursor = cnxn.cursor()

# Load query into dataframe
df=pd.read_sql(query_str,cnxn)

In [3]:
df=df.loc[df['fare_amount']>=0]
df['dropoff_latitude_n']=pd.to_numeric(df['dropoff_latitude'])
df['dropoff_longitude_n']=pd.to_numeric(df['dropoff_longitude'])
df['pickup_latitude_n']=pd.to_numeric(df['pickup_latitude'])
df['pickup_longitude_n']=pd.to_numeric(df['pickup_longitude'])
df=df.drop(columns=['dropoff_latitude','dropoff_longitude','pickup_latitude','pickup_longitude'])

In [4]:
boundary={'min_lng':-74.263242,
              'min_lat':40.573143,
              'max_lng':-72.986532, 
              'max_lat':41.709555}

#We will mark the outlier locations as 1 and remove them for further analysi
df.loc[~((df.pickup_longitude_n >= boundary['min_lng'] ) & (df.pickup_longitude_n <= boundary['max_lng']) &
            (df.pickup_latitude_n >= boundary['min_lat']) & (df.pickup_latitude_n <= boundary['max_lat']) &
            (df.dropoff_longitude_n >= boundary['min_lng']) & (df.dropoff_longitude_n <= boundary['max_lng']) &
            (df.dropoff_latitude_n >=boundary['min_lat']) & (df.dropoff_latitude_n <= boundary['max_lat'])),'is_outlier_loc']=1
df.loc[((df.pickup_longitude_n >= boundary['min_lng'] ) & (df.pickup_longitude_n <= boundary['max_lng']) &
            (df.pickup_latitude_n >= boundary['min_lat']) & (df.pickup_latitude_n <= boundary['max_lat']) &
            (df.dropoff_longitude_n >= boundary['min_lng']) & (df.dropoff_longitude_n <= boundary['max_lng']) &
            (df.dropoff_latitude_n >=boundary['min_lat']) & (df.dropoff_latitude_n <= boundary['max_lat'])),'is_outlier_loc']=0

print("Outlier vs Non Outlier Counts")
print(df['is_outlier_loc'].value_counts())

# Let us drop rows, where location is outlier
df=df.loc[df['is_outlier_loc']==0]
df.drop(['is_outlier_loc'],axis=1,inplace=True)

Outlier vs Non Outlier Counts
0.0    1698334
1.0       5603
Name: is_outlier_loc, dtype: int64


In [5]:
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)

nyc_airports={'JFK':{'min_lng':-73.8352,
     'min_lat':40.6195,
     'max_lng':-73.7401, 
     'max_lat':40.6659},
              
    'EWR':{'min_lng':-74.1925,
            'min_lat':40.6700, 
            'max_lng':-74.1531, 
            'max_lat':40.7081

        },
    'LaGuardia':{'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
        
    }

}
def isAirport(latitude,longitude,airport_name='JFK'):
    
    if latitude>=nyc_airports[airport_name]['min_lat'] and latitude<=nyc_airports[airport_name]['max_lat'] and longitude>=nyc_airports[airport_name]['min_lng'] and longitude<=nyc_airports[airport_name]['max_lng']:
        return 1
    else:
        return 0

In [6]:
def coordinates2distance(lat1,lat2,long1,long2):
    R=6373.0 #radius of earth in km
    p = 0.017453292519943295 # Pi/180 to convert to radians
    lat1=lat1*p
    lat2=lat2*p
    long1=long1*p
    long2=long2*p
    dlon=(long2-long1)
    dlat=(lat2-lat1)
    a=(np.sin(dlat/2)**2)+(np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2)
    c=2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    return R*c

In [7]:
df['distance']=coordinates2distance(df['pickup_latitude_n'].values,
                                    df['dropoff_latitude_n'].values,
                                    df['pickup_longitude_n'].values,
                                    df['dropoff_longitude_n'].values)

In [8]:
nyc_boroughs={
    'manhattan':{
        'min_lng':-74.0479,
        'min_lat':40.6829,
        'max_lng':-73.9067,
        'max_lat':40.8820
    },
    
    'queens':{
        'min_lng':-73.9630,
        'min_lat':40.5431,
        'max_lng':-73.7004,
        'max_lat':40.8007

    },

    'brooklyn':{
        'min_lng':-74.0421,
        'min_lat':40.5707,
        'max_lng':-73.8334,
        'max_lat':40.7395

    },

    'bronx':{
        'min_lng':-73.9339,
        'min_lat':40.7855,
        'max_lng':-73.7654,
        'max_lat':40.9176

    },

    'staten_island':{
        'min_lng':-74.2558,
        'min_lat':40.4960,
        'max_lng':-74.0522,
        'max_lat':40.6490

    }
    
    
    
}

In [9]:
def getBorough(lat,lng):
    
    locs=nyc_boroughs.keys()
    for loc in locs:
        if lat>=nyc_boroughs[loc]['min_lat'] and lat<=nyc_boroughs[loc]['max_lat'] and lng>=nyc_boroughs[loc]['min_lng'] and lng<=nyc_boroughs[loc]['max_lng']:
            return loc
    return 'others'

In [10]:
lower_manhattan_boundary={'min_lng': -74.0194,
                          'min_lat':40.6997,
                          'max_lng':-73.9716,
                          'max_lat':40.7427}

def isLowerManhattan(lat,lng):
    if lat>=lower_manhattan_boundary['min_lat'] and lat<=lower_manhattan_boundary['max_lat'] and lng>=lower_manhattan_boundary['min_lng'] and lng<=lower_manhattan_boundary['max_lng']:
        return 1
    else:
        return 0

In [11]:
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]

In [12]:
df['pickup_datetime']=pd.to_datetime(df['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
df['pickup_date']=  df['pickup_datetime'].dt.date
df['pickup_day']= df['pickup_datetime'].apply(lambda x:x.day)
df['pickup_hour']= df['pickup_datetime'].apply(lambda x:x.hour)
df['pickup_day_of_week']= df['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
df['pickup_month']= df['pickup_datetime'].apply(lambda x:x.month)
df['pickup_year']= df['pickup_datetime'].apply(lambda x:x.year)
df['is_pickup_JFK']= df.apply(lambda row:isAirport(row['pickup_latitude_n'],row['pickup_longitude_n'],'JFK'),axis=1)
df['is_dropoff_JFK']= df.apply(lambda row:isAirport(row['dropoff_latitude_n'],row['dropoff_longitude_n'],'JFK'),axis=1)
df['is_pickup_EWR']= df.apply(lambda row:isAirport(row['pickup_latitude_n'],row['pickup_longitude_n'],'EWR'),axis=1)
df['is_dropoff_EWR']= df.apply(lambda row:isAirport(row['dropoff_latitude_n'],row['dropoff_longitude_n'],'EWR'),axis=1)
df['is_pickup_la_guardia']= df.apply(lambda row:isAirport(row['pickup_latitude_n'],row['pickup_longitude_n'],'LaGuardia'),axis=1)
df['is_dropoff_la_guardia']= df.apply(lambda row:isAirport(row['dropoff_latitude_n'],row['dropoff_longitude_n'],'LaGuardia'),axis=1)
df['pickup_latitude_round3']= df['pickup_latitude_n'].apply(lambda x:round(x,3))
df['pickup_longitude_round3']= df['pickup_longitude_n'].apply(lambda x:round(x,3))
df['dropoff_latitude_round3']= df['dropoff_latitude_n'].apply(lambda x:round(x,3))
df['dropoff_longitude_round3']= df['dropoff_longitude_n'].apply(lambda x:round(x,3))
df['pickup_borough']= df.apply(lambda row:getBorough(row['pickup_latitude_n'],row['pickup_longitude_n']),axis=1)
df['dropoff_borough']= df.apply(lambda row:getBorough(row['dropoff_latitude_n'],row['dropoff_longitude_n']),axis=1)
df['is_pickup_lower_manhattan']= df.apply(lambda row:isLowerManhattan(row['pickup_latitude_n'],row['pickup_longitude_n']),axis=1)
df['is_dropoff_lower_manhattan']= df.apply(lambda row:isLowerManhattan(row['dropoff_latitude_n'],row['dropoff_longitude_n']),axis=1)
df['pickup_day_of_week']= df['pickup_day_of_week'].apply(lambda x:encodeDays(x))
print("Shape of  df data",  df.shape)

Shape of  df data (1698334, 28)


In [ ]:
df.to_csv('nyc_data_clean_and_features.csv',index=False)